# 代码随想录算法训练营第二天｜977.有序数组的平方 ，209.长度最小的子数组 ，59.螺旋矩阵II

## 977.有序数组的平方 

题目建议： 本题关键在于理解双指针思想 

题目链接：https://leetcode.com/problems/squares-of-a-sorted-array/  
文章讲解：https://programmercarl.com/0977.%E6%9C%89%E5%BA%8F%E6%95%B0%E7%BB%84%E7%9A%84%E5%B9%B3%E6%96%B9.html  
视频讲解： https://www.bilibili.com/video/BV1QB4y1D7ep 

__核心__：负数的平方会插到正数平方的队列中，需要比较确定插入位置。  
第一想法：找到负数和正数的分界点，然后指针从分界点向两端移动，按从小到大顺序插入。  
    问题：实现过于复杂，容易出错；特殊情况过多（全是负数、全是正数、一端结束后的处理）。  
暴力解法：直接计算平方，然后排序。 `O(nlogn + n)`  
正解思路：从**两端开始**比较，**从大到小**插入到数组的尾部。

In [11]:
from typing import List
class Solution:
    def sortedSquares(self, nums: List[int]) -> List[int]:
        l, r, i = 0, len(nums)-1, len(nums)-1
        res = [float('inf')] * len(nums) # 需要提前定义列表，存放结果
        while l <= r:
            if nums[l] ** 2 < nums[r] ** 2: # 左右边界进行对比，找出最大值
                res[i] = nums[r] ** 2
                r -= 1 # 右指针往左移动
            else:
                res[i] = nums[l] ** 2
                l += 1 # 左指针往右移动
            i -= 1 # 存放结果的指针需要往前平移一位
        return res


## 209.长度最小的子数组

题目建议： 本题关键在于理解滑动窗口 

题目链接：https://leetcode.com/problems/minimum-size-subarray-sum/  
文章讲解：https://programmercarl.com/0209.%E9%95%BF%E5%BA%A6%E6%9C%80%E5%B0%8F%E7%9A%84%E5%AD%90%E6%95%B0%E7%BB%84.html  
视频讲解：https://www.bilibili.com/video/BV1tZ4y1q7XE


注意：Subarray是连续的子数组，所以滑动窗口的核心是连续性。  
暴力解法：双重循环，从小到大找出所有的子数组，当子数组的和大于等于目标值时返回。`O(n^2)`
#### `O(n)`解法：
1. 外循环：移动右指针，添加元素
2. 内循环：移动左指针，找到最小的子数组
3. 长度：`right - left + 1`：左闭右闭区间

In [ ]:
class Solution:
    def minSubArrayLen(self, s: int, nums: List[int]) -> int:
        l = len(nums)
        left = 0
        right = 0
        min_len = float('inf')
        cur_sum = 0 #当前的累加值
        
        while right < l:
            cur_sum += nums[right]
            
            while cur_sum >= s: # 当前累加值大于目标值
                min_len = min(min_len, right - left + 1)
                cur_sum -= nums[left]
                left += 1
            
            right += 1
        
        return min_len if min_len != float('inf') else 0

#### 尝试：
1. 只移动右指针，找到第一个满足条件的子数组，返回0如果没有找到。
2. 子数组的和大于等于目标值时，移动左指针，找到最小的子数组，更新最小值；小于目标值时，移动右指针。  
缺点：特殊情况处理复杂，容易出错。  
改进处：将左右指针的移动分开判断，因为左指针的移动是有条件的，右指针的移动是无条件的（必须遍历）。

In [ ]:
class Solution:
    def minSubArrayLen(self, target: int, nums: List[int]) -> int:
        total, tail, head,size, diff = 0,0,0,0,0
        while total < target and tail < len(nums):
            total += nums[tail]
            tail += 1
            size += 1
        if tail == len(nums) and total < target:
            return 0
        end = False
        while end == False or diff != tail - head:
            diff = tail - head
            if total >= target:
                size = min(size, diff)
                total -= nums[head]
                head += 1
            elif total < target:
                if tail < len(nums):
                    total += nums[tail]
                    tail += 1
                else:
                    end = True
        return size

提示的`O(nlogn)`解法：待补充

## 59.螺旋矩阵II

题目建议：  本题关键还是在转圈的逻辑，在二分搜索中提到的区间定义，在这里又用上了。 

题目链接：https://leetcode.com/problems/spiral-matrix-ii/  
文章讲解：https://programmercarl.com/0059.%E8%9E%BA%E6%97%8B%E7%9F%A9%E9%98%B5II.html  
视频讲解：https://www.bilibili.com/video/BV1SL4y1N7mV/


#### 二分法，循环不变量
1. 按圈数循环，每次循环填充一圈：圈数为n//2，n为奇数时，最后一个位置单独填充
2. 每次循环按右下左上的顺序填充，每个循环填充`n-1-圈数`个位置（每圈一共有'4*(边长-1)'个格子）
3. 每圈起始位置沿主对角线移动
4. 不随填充位置变化指针 --> 减少赋值次数（本题影响不大？）


In [9]:
class Solution:
    def generateMatrix(self, n: int) -> List[List[int]]:
        nums = [[0] * n for _ in range(n)]
        startx, starty = 0, 0               # 起始点
        loop, mid = n // 2, n // 2          # 迭代次数、n为奇数时，矩阵的中心点
        count = 1                           # 计数

        for offset in range(1, loop + 1) :      # 每循环一层偏移量加1，偏移量从1开始
            for i in range(starty, n - offset) :    # 从左至右，左闭右开
                nums[startx][i] = count
                count += 1
            for i in range(startx, n - offset) :    # 从上至下
                nums[i][n - offset] = count
                count += 1
            for i in range(n - offset, starty, -1) : # 从右至左
                nums[n - offset][i] = count
                count += 1
            for i in range(n - offset, startx, -1) : # 从下至上
                nums[i][starty] = count
                count += 1                
            startx += 1         # 更新起始点
            starty += 1

        if n % 2 != 0 :			# n为奇数时，填充中心点
            nums[mid][mid] = count 
        return nums

0

#### 自己的尝试：
1. 外循环条件：填充值
2. 按右下左上的顺序填充至边界或者已经填充过的位置，每次走一圈
3. 单独填充最后一个位置（原因：先填充后移动，判断循环条件时没进行填充）

In [8]:
from typing import List
class Solution:
    def generateMatrix(self, n: int) -> List[List[int]]:
        matrix = [[0 for _ in range(n)] for _ in range(n)]
        i, j, k = 0,0,1
        while k < (n**2):
            while j < n - 1 and matrix[i][j+1] == 0:
                matrix[i][j] = k
                j += 1
                k += 1
            while i < n - 1 and matrix[i+1][j] == 0:
                matrix[i][j] = k
                i += 1
                k += 1
            while j > 0 and matrix[i][j-1] == 0:
                matrix[i][j] = k
                j -= 1
                k += 1
            while i > 0 and matrix[i-1][j] == 0:
                matrix[i][j] = k
                i -= 1
                k += 1
        matrix[i][j] = k
        return matrix

#### AI作答：
1. 存储方向指针（0，1，-1）
2. 如果下一个位置超出边界（用模避免outOfBounds)或者已经填充过（python直接用非0为True)，改变方向（正交：交换取负）

In [ ]:
class Solution:
    def generateMatrix(self, n: int) -> List[List[int]]:
        matrix = [[0] * n for _ in range(n)]
        i, j, di, dj = 0, 0, 0, 1  # di, dj are directions for i and j
        for k in range(1, n * n + 1):
            matrix[i][j] = k
            # Check if the next cell is out of bounds or already filled
            if matrix[(i + di) % n][(j + dj) % n]:
                # Change direction: right -> down -> left -> up
                di, dj = dj, -di
            i += di
            j += dj
        return matrix

## 总结 

文章链接：https://programmercarl.com/%E6%95%B0%E7%BB%84%E6%80%BB%E7%BB%93%E7%AF%87.html 
